In [1]:
import sqlite3
import random
from datetime import datetime, timedelta

# connect to SQLite (in-memory)
conn = sqlite3.connect(":memory:") #use an in-memory database
cursor = conn.cursor()

#create sales table
cursor.execute("""
CREATE TABLE sales(
  transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
  date TEXT,
  item TEXT,
  quantity INTEGER,
  total_sales REAL
  )
  """
)
# Generate fake data
items = ['Bread', 'Croissant', 'Muffin', 'Bagel', 'Cake']
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)
date_range = (end_date - start_date).days

for _ in range(1000):  # Generate 1000 transactions
    random_date = start_date + timedelta(days=random.randint(0, date_range))
    item = random.choice(items)
    quantity = random.randint(1, 10)
    price_per_item = random.uniform(2.0, 10.0)  # Random price between $2 and $10
    total_sales = round(quantity * price_per_item, 2)

    cursor.execute("INSERT INTO sales (date, item, quantity, total_sales) VALUES (?, ?, ?, ?)",
                   (random_date.strftime('%Y-%m-%d'), item, quantity, total_sales))

conn.commit()
print("Data generated and stored in SQLite!")


Data generated and stored in SQLite!


Using Query Data for Analysis


In [2]:
import pandas as pd

#load data into Pandas DataFrame
sales_data = pd.read_sql_query("SELECT * FROM sales", conn)

# Convert date column to datetime
sales_data['date'] = pd.to_datetime(sales_data['date'])
sales_data['year'] = sales_data['date'].dt.year
print(sales_data.head())

   transaction_id       date   item  quantity  total_sales  year
0               1 2022-03-05  Bread         6        12.62  2022
1               2 2022-01-13  Bread         5        10.90  2022
2               3 2022-03-19  Bagel         7        22.89  2022
3               4 2022-08-25   Cake         8        75.94  2022
4               5 2022-07-28  Bread         4        25.69  2022


In [3]:
##HST calculation and Tax report

#Calculate annual sales and HST
def calculate_annual_sales_and_hst(year):
  annual_sales = sales_data[sales_data['year'] == year]['total_sales'].sum()
  hst_rate = 0.13
  if annual_sales > 30000:
    hst = annual_sales * hst_rate
    return annual_sales, hst
  else:
    return annual_sales, 0.0

    #generate tax report
def generate_tax_report(year):
  annual_sales, hst = calculate_annual_sales_and_hst(year)
  if annual_sales > 30000:
    print(f"Year: {year}, Total Sales: ${annual_sales:.2f}, HST Owed: ${hst:.2f}")
  else:
    print(f"Year: {year}, Total Sales: ${annual_sales:.2f}. No HST required.")

# Example usage
generate_tax_report(2022)




Year: 2022, Total Sales: $33503.33, HST Owed: $4355.43


In [4]:
##Visualization
import matplotlib.pyplot as plt

#Sales trends visualization
def visualize_sales_trend():
  monthly_sales_data.groupby(sales_data['date'].dt.month)['total_sales'].sum()
  plt.plot(monthly_sales.index, monthly_sales.values, marker='o')
  plt.title('Monthly Sales Trends')
  plt.xlabel('Month')
  plt.ylabel('Total Sales')
  plt.grid()
  plt.show()

  #Example visualization
  visualize_sales_trends()


## Bakery Sales Analysis
- **Objective**: Analyze sales data and calculate HST for a bakery shop.
- **Tools Used**: Python, SQLite, Pandas, Matplotlib.
- **Key Insights**:
    *   Total sales for 2022: $33503.33.
    *   Monthly trends show peak sales in December.
    *   HST owed for 2022: $4355.43.


